In [1]:
# Устанавливаем необходимые библиотеки
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.6 MB/s eta 0:00:00


In [2]:
# Подключаем хранилище с файлами
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
import torch
from transformers import TrainingArguments, Trainer
#Проверяем, что мы работаем на GPU 
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("ЗАПУСТИЛОСЬ БЕЗ ВИДЕОКАРТЫ")
    

In [50]:
#Считываем jsonl файл
import json

with open('/content/drive/MyDrive/CupIT/ranking_train.jsonl', 'r') as json_file:
    json_list = list(json_file)
listOfPostsAsDicts = [json.loads(json_str) for json_str in json_list]
justcomments = [post["comments"] for post in listOfPostsAsDicts]
justcomments = [comment for comments in justcomments for comment in comments]
print(len(justcomments))

440535


In [51]:
#Создаём таблицу с исходными данными
dff = pd.DataFrame(justcomments)
posts = [[post["text"]]*5 for post in listOfPostsAsDicts]
posts = [subpost for post in posts for subpost in post]
dff["post"] = posts
cols = ["post","text","score"]
dff = dff[cols]
dff.columns = ["post","comment","labels"]
dff

,post,comment,labels
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called &#x27;...,0
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone&#x27;s Ren...,3


In [ ]:
# 90% Выборки зарезервируем для обучения, на оставшихся значениях проведём тест точности
df = dff[:400000]
# Объёдиняем тексты постов и комментариев в единые строки с разделителем [SEP]
df["text"] = [post + " [SEP] " + comment for post,comment in zip(df["post"],df["comment"])]

df = df.drop(columns=["post","comment"])
df["labels"] = [float(label) for label in df["labels"]]
df

<ipython-input-7-269110202ce4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = [post + " [SEP] " + comment for post,comment in zip(df["post"],df["comment"])]


,labels,text
0,0.0,How many summer Y Combinator fundees decided n...
1,1.0,How many summer Y Combinator fundees decided n...
2,2.0,How many summer Y Combinator fundees decided n...
3,3.0,How many summer Y Combinator fundees decided n...
4,4.0,How many summer Y Combinator fundees decided n...
...,...,...
399995,0.0,The Operating System That Can Protect You Even...
399996,1.0,The Operating System That Can Protect You Even...
399997,2.0,The Operating System That Can Protect You Even...
399998,3.0,The Operating System That Can Protect You Even...


In [ ]:
#Разобъём данные на датасеты для тренировки и валидации
dataset = Dataset.from_pandas(df,preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 

dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 280000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 120000
    })
})

In [5]:
#В качестве модели будем использовать DistillBert т.к. время тренировки ограничено и нужен быстрый результат
#Очистку данных перед тренировкой проводить не будем, так как DistillBert специально тренировался на "грязных" данных
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
#Определяем дополнительную функцию для более простой токенизации
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)



Сначала мы начали делать модель классификации для решения поставленной задачи, которая выдавала бы комментариям класс от 0 до 4. Мы её даже доделали на обычном Bert, однако после нескольких часов тренировки и первых тестов, мы переосмыслили свой подход и пришли к выводу, что проще и эффективнее будет использовать регрессионную модель, так как сортировать континуальные значения очень легко, одинаковых прогнозов для разных текстов не возникает. Кроме того, такой подход позволяет использовать модель для ранжирования неограниченного количества комментариев, а не только 5.

In [ ]:
#Загружаем DistillBert
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
# По причине того что модель у нас регрессионная, нам будет сложно вычислять отклонение после сортировки, поэтому для тренировки ограничимся среднеквадратичным
from datasets import load_metric

mean_squared_error = load_metric("mse")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mean_squared_error.add_batch(references=labels, predictions=predictions)
    return mean_squared_error.compute(squared=False)
compute_metrics(([1,2,3],[1,2,5]))

<ipython-input-10-81fb824f0025>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  mean_squared_error = load_metric("mse")


{'mse': 1.1547005383792515}

In [ ]:
#Начинаем тренировку. 3 эпохи, так как время ограничено.
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/CupIT/test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'steps',
                                  load_best_model_at_end=False
                                                                    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
trainer.train("/content/drive/MyDrive/CupIT/test_trainer/checkpoint-34000")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/16500 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Mse
2,1.651800,1.652335,1.285432
3,1.655500,1.631331,1.277236


TrainOutput(global_step=52500, training_loss=0.5832868443080357, metrics={'train_runtime': 17740.7534, 'train_samples_per_second': 47.349, 'train_steps_per_second': 2.959, 'total_flos': 1.1127063048192e+17, 'train_loss': 0.5832868443080357, 'epoch': 3.0})

In [ ]:
# Сохраняем модель

model.save_pretrained("/content/drive/MyDrive/CupIT/regressionModel")

In [ ]:
# Это для загрузки модели
# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/CupIT/regressionModel")

In [6]:
#Это для загрузки модели с чекпоинта
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/CupIT/checkpoint-52500").to(device)

In [7]:
#Создаём функцию для выполнения прогнозов
from transformers import Trainer
trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df=pd.DataFrame({'text':text})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return [elem[0] for elem in (raw_pred)]

text = ["Four Pownce Invites [SEP] Ok these are out the door.  I have a few more here if anybody would like them.",
        "Four Pownce Invites [SEP] sgoraya at gmail.com\nthx!",
        "Four Pownce Invites [SEP] kbeshay@gmail.comthanks.",
        "Four Pownce Invites [SEP] uprz23@gmail.comThanks.",
        "Four Pownce Invites [SEP] Ok these are out the door.  I have a few more here if anybody would like them."]
pipeline_prediction(text)


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

[2.550453, 3.4011967, 3.4095325, 3.409274, 2.550453]

In [52]:
#Создадим тестовый набор данных
test_df = dff[400000:]
test_df["text"] = [post + " [SEP] " + comment for post,comment in zip(test_df["post"],test_df["comment"])]

test_df = test_df.drop(columns=["post","comment"])
test_df["labels"] = [float(label) for label in test_df["labels"]]
test_df



<ipython-input-52-3dfe5a13b6aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["text"] = [post + " [SEP] " + comment for post,comment in zip(test_df["post"],test_df["comment"])]


,labels,text
400000,0.0,Omaha: Software installer and auto-updater for...
400001,1.0,Omaha: Software installer and auto-updater for...
400002,2.0,Omaha: Software installer and auto-updater for...
400003,3.0,Omaha: Software installer and auto-updater for...
400004,4.0,Omaha: Software installer and auto-updater for...
...,...,...
440530,0.0,Pay your rent with a Credit or Debit card. No ...
440531,1.0,Pay your rent with a Credit or Debit card. No ...
440532,2.0,Pay your rent with a Credit or Debit card. No ...
440533,3.0,Pay your rent with a Credit or Debit card. No ...


In [53]:
predicted_scores = pipeline_prediction(test_df["text"])

Map:   0%|          | 0/40535 [00:00<?, ? examples/s]

In [54]:
#Присовим комментариям целочисленный рейтинг согласно спрогнозированному значению
int_predicted_scores=[]
for i in range(0,len(predicted_scores),5):
  pred5 = predicted_scores[i:i+5]
  int_pred5=pred5[:]
  for j in range(5):
    int_pred5[np.argmin(pred5)] = j
    pred5[np.argmin(pred5)] = float("inf")
  int_predicted_scores.append(int_pred5)
int_predicted_scores[0]

[3, 2, 0, 1, 4]

In [ ]:
#Посмотрим процент соответствия правильных прогнозов
guesses = 0
real = [[0,1,2,3,4] for _ in range(len(int_predicted_scores)//5)]
for p,r in zip(int_predicted_scores, real):
  for pe, re in zip(p,r):
    if pe == re:
      guesses+=1
print(guesses/len(real)/5)


0.3110425663170882


Вот по такой предложенной нами (а конкретно одним человеком, пальцем показывать не будем) метрике мы долгое время смотрели результаты и плакали что ничего не получается.

In [58]:
#Измерим ошибку по предложенной кейсом метрике
import sklearn
sklearn.metrics.ndcg_score([[0,1,2,3,4] for _ in range(len(int_predicted_scores))],int_predicted_scores)

0.8789462936326662

И только в самом конце решив сдавать уже то что есть мы вспомнили что в задании предлагалось использовать ndcg. 

In [19]:
# Теперь заполним тестовый файл
def score(text):
  predicted_scores = pipeline_prediction(text)
  int_predicted_scores=[]
  for i in range(0,len(predicted_scores),5):
    pred5 = predicted_scores[i:i+5]
    int_pred5=pred5[:]
    for j in range(5):
      int_pred5[np.argmin(pred5)] = j
      pred5[np.argmin(pred5)] = float("inf")
    int_predicted_scores.append(int_pred5)
  return int_predicted_scores


import json

# Чтение данных из файла
with open('/content/drive/MyDrive/CupIT/ranking_test.jsonl', 'r') as f:
    data = f.read()

# Преобразование данных в формат json
data = json.loads('[' + data.replace('}\n{', '},\n{') + ']')

# Извлечение текста постов и комментариев
megatext = []
for d in data:
    text = []
    post_text = d['text']
    for comment in d['comments']:
        comment_text = comment['text']
        text.append(f"{post_text} [SEP] {comment_text}")
    megatext.append(text)





Map:   0%|          | 0/70020 [00:00<?, ? examples/s]

In [28]:
# Расчет оценок для каждого поста 
# Манипуляции с мерностью массива из-за того что код писали разные люди и не успели договориться
megatext = [elem for text in megatext for elem in text]
scores = score(megatext)
scoresUnzip = [subelem for elem in scores for subelem in elem]


4

In [34]:
#Записываем данные обратно в новый файл
with open('/content/drive/MyDrive/CupIT/ranking_test.jsonl', 'r') as f:
    data = f.read()

# Преобразование данных в формат json
data = json.loads('[' + data.replace('}\n{', '},\n{') + ']')
# Замена null на соответствующие значения из scores
for i, post in enumerate(data):
    for j, comment in enumerate(post['comments']):
        if comment['score'] is None:
            comment['score'] = scoresUnzip[i][j]


# Запись результата в новый файл
with open('/content/drive/MyDrive/CupIT/ranking_test_filled.jsonl', 'w') as f:
    for d in data:
        f.write(json.dumps(d) + '\n')